In [1]:
!pip install Flask nest_asyncio

In [2]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [2]:
import nest_asyncio
from flask import Flask, render_template_string, request
from threading import Thread

# Apply nest_asyncio to allow Flask to run in Jupyter notebook
nest_asyncio.apply()

app = Flask(__name__)

# Mock data
doctors = {
    "espanol": ["Dr. García", "Dr. Martínez", "Dr. López", "Dr. Hernández", "Dr. Pérez"],
    "german": ["Dr. Müller", "Dr. Schmidt"]
}

hospitals = {
    "espanol": ["Hospital San Juan", "Clínica La Paz", "Hospital Central", "Clínica del Sol", "Hospital Universitario"],
    "german": ["Krankenhaus Berlin", "Klinik München"]
}

# Define the templates
signup_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>StudiMed - Sign Up</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 0; display: flex; justify-content: center; align-items: center; height: 100vh; background-color: #f8f8f8; }
        .container { width: 360px; background: #fff; border: 1px solid #ddd; box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); padding: 20px; border-radius: 10px; text-align: center; }
        h1 { font-size: 24px; margin-bottom: 20px; }
        form { display: flex; flex-direction: column; }
        input[type="text"], input[type="tel"] { padding: 10px; margin-bottom: 10px; border: 1px solid #ccc; border-radius: 5px; font-size: 16px; }
        button { padding: 10px; margin-top: 10px; background-color: #007bff; color: white; border: none; border-radius: 5px; font-size: 16px; cursor: pointer; }
        button:hover { background-color: #0056b3; }
    </style>
</head>
<body>
    <div class="container">
        <h1>StudiMed - Sign Up</h1>
        <form action="/language" method="POST">
            <label for="name">Name:</label>
            <input type="text" id="name" name="name" required><br>
            <label for="phone">Phone Number:</label>
            <input type="tel" id="phone" name="phone" required><br>
            <button type="submit">Enter</button>
        </form>
    </div>
</body>
</html>
'''

language_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>StudiMed - Select Language</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 0; display: flex; justify-content: center; align-items: center; height: 100vh; background-color: #f8f8f8; }
        .container { width: 360px; background: #fff; border: 1px solid #ddd; box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); padding: 20px; border-radius: 10px; text-align: center; }
        h1 { font-size: 24px; margin-bottom: 20px; }
        .carousel { display: flex; overflow-x: scroll; }
        .carousel button { margin: 5px; padding: 10px; background-color: #007bff; color: white; border: none; border-radius: 5px; cursor: pointer; }
        .carousel button:hover { background-color: #0056b3; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Select Language</h1>
        <form action="/selection" method="POST">
            <input type="hidden" name="name" value="{{ name }}">
            <input type="hidden" name="phone" value="{{ phone }}">
            <div class="carousel">
                <button type="submit" name="language" value="espanol">Español</button>
                <button type="submit" name="language" value="german">German</button>
                <button type="submit" name="language" value="french">Français</button>
                <button type="submit" name="language" value="italian">Italiano</button>
                <button type="submit" name="language" value="chinese">中文</button>
                <button type="submit" name="language" value="japanese">日本語</button>
                <button type="submit" name="language" value="korean">한국어</button>
                <button type="submit" name="language" value="portuguese">Português</button>
                <button type="submit" name="language" value="russian">Русский</button>
                <button type="submit" name="language" value="arabic">العربية</button>
            </div>
        </form>
    </div>
</body>
</html>
'''

selection_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>StudiMed - Selection</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 0; display: flex; justify-content: center; align-items: center; height: 100vh; background-color: #f8f8f8; }
        .container { width: 360px; background: #fff; border: 1px solid #ddd; box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); padding: 20px; border-radius: 10px; text-align: center; }
        h1 { font-size: 24px; margin-bottom: 20px; }
        h2 { font-size: 20px; margin-bottom: 20px; }
        form { display: flex; flex-direction: column; }
        button { padding: 10px; margin-top: 10px; background-color: #007bff; color: white; border: none; border-radius: 5px; font-size: 16px; cursor: pointer; }
        button:hover { background-color: #0056b3; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Welcome, {{ name }}!</h1>
        <h2>Select an Option</h2>
        <form action="/appointments/{{ language }}" method="GET">
            <button type="submit">Appointments</button>
        </form>
        <form action="/near_you/{{ language }}" method="GET">
            <button type="submit">Near You</button>
        </form>
        <form action="/queries?language={{ language }}" method="GET">
            <button type="submit">General Queries</button>
        </form>
    </div>
</body>
</html>
'''

appointments_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>StudiMed - Appointments</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 0; display: flex; justify-content: center; align-items: center; height: 100vh; background-color: #f8f8f8; }
        .container { width: 360px; background: #fff; border: 1px solid #ddd; box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); padding: 20px; border-radius: 10px; text-align: center; }
        h1 { font-size: 24px; margin-bottom: 20px; }
        ul { list-style-type: none; padding: 0; }
        li { padding: 10px; background-color: #f1f1f1; margin-bottom: 10px; border-radius: 5px; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Available Doctors</h1>
        <ul>
            {% for doctor in doctors %}
                <li>{{ doctor }}</li>
            {% endfor %}
        </ul>
    </div>
</body>
</html>
'''

near_you_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>StudiMed - Hospitals Near You</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 0; display: flex; justify-content: center; align-items: center; height: 100vh; background-color: #f8f8f8; }
        .container { width: 360px; background: #fff; border: 1px solid #ddd; box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); padding: 20px; border-radius: 10px; text-align: center; }
        h1 { font-size: 24px; margin-bottom: 20px; }
        ul { list-style-type: none; padding: 0; }
        li { padding: 10px; background-color: #f1f1f1; margin-bottom: 10px; border-radius: 5px; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Hospitals Near You</h1>
        <ul>
            {% for hospital in hospitals %}
                <li>{{ hospital }}</li>
            {% endfor %}
        </ul>
    </div>
</body>
</html>
'''

queries_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>StudiMed - General Queries</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 0; display: flex; justify-content: center; align-items: center; height: 100vh; background-color: #f8f8f8; }
        .container { width: 360px; background: #fff; border: 1px solid #ddd; box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); padding: 20px; border-radius: 10px; text-align: center; }
        h1 { font-size: 24px; margin-bottom: 20px; }
        .chat-box { border: 1px solid #ccc; border-radius: 5px; padding: 10px; margin-bottom: 10px; height: 300px; overflow-y: scroll; }
        .chat-input { display: flex; }
        .chat-input input { flex: 1; padding: 10px; border: 1px solid #ccc; border-radius: 5px; margin-right: 10px; }
        .chat-input button { padding: 10px; background-color: #007bff; color: white; border: none; border-radius: 5px; cursor: pointer; }
        .chat-input button:hover { background-color: #0056b3; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Chat Interface</h1>
        <div class="chat-box" id="chat-box">
            <p><strong>StudiMed:</strong> ¿En qué puedo ayudarte hoy?</p>
        </div>
        <div class="chat-input">
            <input type="text" id="chat-input" placeholder="Escribe tu mensaje...">
            <button onclick="sendMessage()">Enviar</button>
        </div>
    </div>
    <script>
        function sendMessage() {
            const input = document.getElementById('chat-input');
            const chatBox = document.getElementById('chat-box');
            const message = input.value;
            if (message) {
                const userMessage = document.createElement('p');
                userMessage.innerHTML = '<strong>Tú:</strong> ' + message;
                chatBox.appendChild(userMessage);
                input.value = '';
                chatBox.scrollTop = chatBox.scrollHeight;

                // Simulate a reply from StudiMed
                setTimeout(() => {
                    const reply = document.createElement('p');
                    reply.innerHTML = '<strong>StudiMed:</strong> Gracias por tu mensaje. Estamos aquí para ayudarte.';
                    chatBox.appendChild(reply);
                    chatBox.scrollTop = chatBox.scrollHeight;
                }, 1000);
            }
        }
    </script>
</body>
</html>
'''

@app.route('/')
def signup():
    return render_template_string(signup_template)

@app.route('/language', methods=['POST'])
def language():
    name = request.form['name']
    phone = request.form['phone']
    return render_template_string(language_template, name=name, phone=phone)

@app.route('/selection', methods=['POST'])
def selection():
    language = request.form['language']
    name = request.form['name']
    phone = request.form['phone']
    return render_template_string(selection_template, language=language, name=name, phone=phone)

@app.route('/appointments/<language>')
def appointments(language):
    return render_template_string(appointments_template, doctors=doctors[language])

@app.route('/near_you/<language>')
def near_you(language):
    return render_template_string(near_you_template, hospitals=hospitals[language])

@app.route('/queries')
def queries():
    language = request.args.get('language')
    return render_template_string(queries_template, language=language)

# Function to run Flask app
def run_app():
    app.run(port=5000)

# Run the Flask app in a separate thread
thread = Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/May/2024 12:36:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2024 12:37:47] "GET / HTTP/1.1" 200 -
